# Optuna Per-Experiment Best Results

This notebook extracts and displays the best hyperparameters found by per-experiment Optuna tuning,
along with their **test-set** evaluation metrics (error rate, macro-F1, ECE).

Test metrics are parsed from `study.log` files since the Optuna objective only records dev macro-F1.

In [123]:
# Configuration
RESULTS_DIR = "../results/optuna/per_experiment"
N_TRIALS = 10  # Trial count to load (None = latest available)

In [124]:
import sys
sys.path.insert(0, "..")

import pandas as pd
from extract_optuna_test_results import extract_results

results = extract_results(RESULTS_DIR, n_trials=N_TRIALS)
print(f"Loaded {len(results)} experiment results")

Loaded 112 experiment results


## Full Results Table

All experiments with best hyperparameters, dev F1, and test metrics.

In [125]:
# Build DataFrame
rows = []
for r in results:
    row = {
        "event": r["event"],
        "budget": r["budget"],
        "seed_set": r["seed_set"],
        "dev_macro_f1": r["dev_macro_f1"],
        "test_macro_f1": r["test_macro_f1"],
        "test_error_rate": r["test_error_rate"],
        "test_ece": r["test_ece"],
    }
    if r.get("best_params"):
        row["lr"] = r["best_params"].get("lr")
        row["batch_size"] = r["best_params"].get("batch_size")
        row["cotrain_epochs"] = r["best_params"].get("cotrain_epochs")
        row["finetune_patience"] = r["best_params"].get("finetune_patience")
        row["weight_decay"] = r["best_params"].get("weight_decay")
        row["warmup_ratio"] = r["best_params"].get("warmup_ratio")
    rows.append(row)

df = pd.DataFrame(rows).sort_values(["event", "budget", "seed_set"]).reset_index(drop=True)

# Format for display
format_dict = {
    "dev_macro_f1": "{:.4f}",
    "test_macro_f1": "{:.4f}",
    "test_error_rate": "{:.2f}",
    "test_ece": "{:.4f}",
    "lr": "{:.6f}",
    "weight_decay": "{:.4f}",
    "warmup_ratio": "{:.4f}",
}
df.style.format(format_dict, na_rep="-")

,event,budget,seed_set,dev_macro_f1,test_macro_f1,test_error_rate,test_ece,lr,batch_size,cotrain_epochs,finetune_patience,weight_decay,warmup_ratio
0,california_wildfires_2018,5,1,0.6306,0.6079,29.16,0.1533,0.000148,16,17,6,0.0979,0.1507
1,california_wildfires_2018,5,2,0.6286,0.6014,27.52,0.1896,0.000140,32,20,10,0.0993,0.1355
2,california_wildfires_2018,5,3,0.6313,0.6242,27.99,0.1788,0.000034,8,6,6,0.0290,0.1010
3,california_wildfires_2018,10,1,0.6272,0.5982,31.14,0.0862,0.000161,32,15,9,0.0142,0.1848
4,california_wildfires_2018,10,2,0.6492,0.6560,26.97,0.1266,0.000025,16,11,9,0.0041,0.0892
5,california_wildfires_2018,10,3,0.6416,0.6376,29.09,0.1073,0.000079,8,7,4,0.0730,0.2660
6,california_wildfires_2018,25,1,0.6302,0.6229,28.54,0.0398,0.000025,8,5,5,0.0353,0.2122
7,california_wildfires_2018,25,2,0.6689,0.6488,29.09,0.1107,0.000072,64,9,10,0.0304,0.0574
8,california_wildfires_2018,25,3,0.6393,0.6418,27.93,0.0599,0.000080,8,12,4,0.0304,0.2110
9,california_wildfires_2018,50,1,0.6491,0.6598,27.45,0.1310,0.000032,16,17,6,0.0581,0.0452


## Summary: Mean Test Macro-F1 and ECE by Event and Budget

Averaged over 3 seed sets per (event, budget) combination.

In [126]:
# Pivot table: event x budget, values = mean test_macro_f1 over seeds
pivot_f1 = df.pivot_table(
    values="test_macro_f1",
    index="event",
    columns="budget",
    aggfunc="mean",
)
pivot_f1["mean"] = pivot_f1.mean(axis=1)
pivot_f1.loc["mean"] = pivot_f1.mean(axis=0)

print("Test Macro-F1")
display(pivot_f1.style.format("{:.4f}", na_rep="-").background_gradient(cmap="YlGn", axis=None))

# Pivot table: event x budget, values = mean test_ece over seeds
pivot_ece = df.pivot_table(
    values="test_ece",
    index="event",
    columns="budget",
    aggfunc="mean",
)
pivot_ece["mean"] = pivot_ece.mean(axis=1)
pivot_ece.loc["mean"] = pivot_ece.mean(axis=0)

print("\nTest ECE (lower is better)")
display(pivot_ece.style.format("{:.4f}", na_rep="-").background_gradient(cmap="YlOrRd", axis=None))

Test Macro-F1


budget,5,10,25,50,mean
event,,,,,
california_wildfires_2018,0.6112,0.6306,0.6378,0.6615,0.6353
canada_wildfires_2016,0.5656,0.5956,0.5962,0.5807,0.5845
cyclone_idai_2019,0.5746,0.6005,0.6095,0.5869,0.5929
hurricane_dorian_2019,0.5606,0.5764,0.5768,0.5952,0.5772
hurricane_florence_2018,0.6622,0.6665,0.6835,0.6713,0.6709
hurricane_harvey_2017,0.6255,0.6442,0.6686,0.6745,0.6532
hurricane_irma_2017,0.6207,0.6351,0.6526,0.6543,0.6407
hurricane_maria_2017,0.6461,0.6451,0.6654,0.6660,0.6556
kaikoura_earthquake_2016,0.6748,0.6946,0.6976,0.6988,0.6915



Test ECE (lower is better)


budget,5,10,25,50,mean
event,,,,,
california_wildfires_2018,0.1739,0.1067,0.0701,0.0927,0.1109
canada_wildfires_2016,0.1065,0.1067,0.0873,0.1091,0.1024
cyclone_idai_2019,0.0867,0.0894,0.1035,0.1065,0.0965
hurricane_dorian_2019,0.2321,0.2059,0.1341,0.1392,0.1778
hurricane_florence_2018,0.1704,0.1311,0.1069,0.1020,0.1276
hurricane_harvey_2017,0.2401,0.1582,0.1593,0.1306,0.1721
hurricane_irma_2017,0.2746,0.2137,0.1408,0.1705,0.1999
hurricane_maria_2017,0.2185,0.1894,0.1289,0.1136,0.1626
kaikoura_earthquake_2016,0.1464,0.1422,0.1192,0.1060,0.1285


## Summary: Mean Test Macro-F1 by Event

In [127]:
by_event = df.groupby("event").agg(
    test_macro_f1_mean=("test_macro_f1", "mean"),
    test_macro_f1_std=("test_macro_f1", "std"),
    test_ece_mean=("test_ece", "mean"),
    dev_macro_f1_mean=("dev_macro_f1", "mean"),
    test_error_rate_mean=("test_error_rate", "mean"),
    count=("test_macro_f1", "count"),
).sort_values("test_macro_f1_mean", ascending=False)

by_event.style.format({
    "test_macro_f1_mean": "{:.4f}",
    "test_macro_f1_std": "{:.4f}",
    "test_ece_mean": "{:.4f}",
    "dev_macro_f1_mean": "{:.4f}",
    "test_error_rate_mean": "{:.2f}",
}, na_rep="-").background_gradient(cmap="YlGn", subset=["test_macro_f1_mean"])

,test_macro_f1_mean,test_macro_f1_std,test_ece_mean,dev_macro_f1_mean,test_error_rate_mean,count
event,,,,,,
kaikoura_earthquake_2016,0.6915,0.0300,0.1285,0.6842,26.67,12
hurricane_florence_2018,0.6709,0.0147,0.1276,0.6922,24.88,12
hurricane_maria_2017,0.6556,0.0165,0.1626,0.6826,31.19,12
hurricane_harvey_2017,0.6532,0.0228,0.1721,0.6644,30.55,12
hurricane_irma_2017,0.6407,0.0181,0.1999,0.6423,33.52,12
california_wildfires_2018,0.6353,0.0239,0.1109,0.6424,28.27,12
cyclone_idai_2019,0.5929,0.0310,0.0965,0.5375,27.28,12
canada_wildfires_2016,0.5845,0.0215,0.1024,0.6956,24.48,12
hurricane_dorian_2019,0.5772,0.0177,0.1778,0.5987,36.61,12


## Summary: Mean Test Macro-F1 by Budget

In [128]:
by_budget = df.groupby("budget").agg(
    test_macro_f1_mean=("test_macro_f1", "mean"),
    test_macro_f1_std=("test_macro_f1", "std"),
    test_ece_mean=("test_ece", "mean"),
    dev_macro_f1_mean=("dev_macro_f1", "mean"),
    test_error_rate_mean=("test_error_rate", "mean"),
    count=("test_macro_f1", "count"),
)

by_budget.style.format({
    "test_macro_f1_mean": "{:.4f}",
    "test_macro_f1_std": "{:.4f}",
    "test_ece_mean": "{:.4f}",
    "dev_macro_f1_mean": "{:.4f}",
    "test_error_rate_mean": "{:.2f}",
}, na_rep="-").background_gradient(cmap="YlGn", subset=["test_macro_f1_mean"])

,test_macro_f1_mean,test_macro_f1_std,test_ece_mean,dev_macro_f1_mean,test_error_rate_mean,count
budget,,,,,,
5,0.6101,0.0451,0.1875,0.6317,29.76,30
10,0.6291,0.0421,0.1519,0.6383,29.52,28
25,0.6431,0.0418,0.1167,0.6559,28.92,27
50,0.6432,0.0456,0.1189,0.6663,28.90,27


In [104]:

# ── Excel export: CoTrain results in the paper-style layout ──────────────────
#
# Column structure:
#   A          : "CoTrain"  (merged over all 8 data rows)
#   B          : budget label ("5 lb/class", …, "50 lb/class")  (merged 2 rows per budget)
#   C–D merged : "Metrics/Event Name Average"
#     C        : metric name ("Macro F1" / "ECE")
#     D        : grand average across all events & seeds
#   E … end    : one group of 3 columns per event  (Set 1 | Set 2 | Set 3)
# ─────────────────────────────────────────────────────────────────────────────

import openpyxl
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils import get_column_letter

OUTPUT_PATH = "../results/cotrain_results_summary.xlsx"

EVENT_DISPLAY = {
    "california_wildfires_2018": "California Wildfires 2018",
    "canada_wildfires_2016":     "Canada Wildfires 2016",
    "cyclone_idai_2019":         "Cyclone Idai 2019",
    "hurricane_dorian_2019":     "Hurricane Dorian 2019",
    "hurricane_florence_2018":   "Hurricane Florence 2018",
    "hurricane_harvey_2017":     "Hurricane Harvey 2017",
    "hurricane_irma_2017":       "Hurricane Irma 2017",
    "hurricane_maria_2017":      "Hurricane Maria 2017",
    "kaikoura_earthquake_2016":  "Kaikoura Earthquake 2016",
}

BUDGETS    = [5, 10, 25, 50]
SEEDS      = [1, 2, 3]
METRICS    = [("Macro F1", "test_macro_f1"), ("ECE", "test_ece")]
events_ord = sorted(df["event"].unique())          # alphabetical, stable order

# ── Layout constants ──────────────────────────────────────────────────────────
COL_MODEL  = 1                                     # A
COL_BUDGET = 2                                     # B
COL_METRIC = 3                                     # C  \  merged header "Metrics/Event Name Average"
COL_AVG    = 4                                     # D  /
COL_DATA   = 5                                     # E  first event-Set-1 column

N_EVENTS      = len(events_ord)
TOTAL_COLS    = COL_DATA + N_EVENTS * len(SEEDS) - 1
HEADER_ROWS   = 2
DATA_ROWS     = len(BUDGETS) * len(METRICS)        # 8
TOTAL_ROWS    = HEADER_ROWS + DATA_ROWS

# ── Styles ────────────────────────────────────────────────────────────────────
BLUE_HEX     = "4472C4"
LT_BLUE_HEX  = "D9E1F2"
LT_GREEN_HEX = "E2EFDA"
LT_AMBER_HEX = "FFF2CC"

def solid(hex_col):
    return PatternFill(start_color=hex_col, end_color=hex_col, fill_type="solid")

def thin_border():
    s = Side(style="thin", color="BFBFBF")
    return Border(left=s, right=s, top=s, bottom=s)

def thick_border():
    s = Side(style="medium", color="000000")
    return Border(left=s, right=s, top=s, bottom=s)

def center(wrap=False):
    return Alignment(horizontal="center", vertical="center", wrap_text=wrap)

def left_middle():
    return Alignment(horizontal="left", vertical="center")

NUM_FMT = "0.000"

# ── Workbook ──────────────────────────────────────────────────────────────────
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "CoTrain Results"

# freeze panes: keep header + row-label cols visible while scrolling
ws.freeze_panes = ws.cell(row=HEADER_ROWS + 1, column=COL_DATA)

# ── Header Row 1 ─────────────────────────────────────────────────────────────
# Cols A–B: blue background, no text
for c in [COL_MODEL, COL_BUDGET]:
    ws.cell(row=1, column=c).fill = solid(BLUE_HEX)

# Cols C–D: merged "Metrics/Event Name Average"
ws.merge_cells(start_row=1, start_column=COL_METRIC,
               end_row=1,   end_column=COL_AVG)
cell = ws.cell(row=1, column=COL_METRIC)
cell.value     = "Metrics/Event Name Average"
cell.font      = Font(bold=True, color="FFFFFF", size=10)
cell.fill      = solid(BLUE_HEX)
cell.alignment = center(wrap=True)

# Cols E+: one merged cell per event (spans 3 seed columns)
for i, ev in enumerate(events_ord):
    sc = COL_DATA + i * len(SEEDS)
    ec = sc + len(SEEDS) - 1
    ws.merge_cells(start_row=1, start_column=sc, end_row=1, end_column=ec)
    cell = ws.cell(row=1, column=sc)
    cell.value     = EVENT_DISPLAY.get(ev, ev)
    cell.font      = Font(bold=True, color="FFFFFF", size=9)
    cell.fill      = solid(BLUE_HEX)
    cell.alignment = center(wrap=True)

# ── Header Row 2 ─────────────────────────────────────────────────────────────
for c in [COL_MODEL, COL_BUDGET]:
    ws.cell(row=2, column=c).fill = solid(BLUE_HEX)

# Col C: "Metric"
cell = ws.cell(row=2, column=COL_METRIC)
cell.value     = "Metric"
cell.font      = Font(bold=True, size=9)
cell.fill      = solid(LT_BLUE_HEX)
cell.alignment = center()

# Col D: "Average"
cell = ws.cell(row=2, column=COL_AVG)
cell.value     = "Average"
cell.font      = Font(bold=True, size=9)
cell.fill      = solid(LT_BLUE_HEX)
cell.alignment = center()

# Event seed sub-headers
for i in range(N_EVENTS):
    for j, s in enumerate(SEEDS):
        col = COL_DATA + i * len(SEEDS) + j
        cell = ws.cell(row=2, column=col)
        cell.value     = f"Set {s}"
        cell.font      = Font(bold=True, size=9)
        cell.fill      = solid(LT_BLUE_HEX)
        cell.alignment = center()

# ── Data Rows ─────────────────────────────────────────────────────────────────
# "CoTrain" merged over all 8 data rows
ws.merge_cells(start_row=HEADER_ROWS + 1, start_column=COL_MODEL,
               end_row=TOTAL_ROWS,        end_column=COL_MODEL)
cell = ws.cell(row=HEADER_ROWS + 1, column=COL_MODEL)
cell.value     = "CoTrain"
cell.font      = Font(bold=True, size=10)
cell.alignment = center(wrap=False)

for b_idx, budget in enumerate(BUDGETS):
    r_macro = HEADER_ROWS + 1 + b_idx * 2      # row for Macro F1
    r_ece   = r_macro + 1                       # row for ECE
    bdf     = df[df["budget"] == budget]

    # Budget label: merged across Macro F1 + ECE rows
    ws.merge_cells(start_row=r_macro, start_column=COL_BUDGET,
                   end_row=r_ece,     end_column=COL_BUDGET)
    cell = ws.cell(row=r_macro, column=COL_BUDGET)
    cell.value     = f"{budget} lb/class"
    cell.font      = Font(bold=True, size=9)
    cell.alignment = center()
    # alternate shading per budget block
    bg = solid("F2F2F2") if b_idx % 2 == 0 else solid("FFFFFF")
    cell.fill = bg

    for m_idx, (mname, mcol) in enumerate(METRICS):
        row  = r_macro + m_idx
        fill = solid(LT_GREEN_HEX) if mname == "Macro F1" else solid(LT_AMBER_HEX)

        # Col C: metric name
        cell = ws.cell(row=row, column=COL_METRIC)
        cell.value     = mname
        cell.font      = Font(size=9)
        cell.fill      = fill
        cell.alignment = left_middle()
        cell.border    = thin_border()

        # Col D: grand average across all (event, seed) combinations
        avg_val = bdf[mcol].mean()
        cell = ws.cell(row=row, column=COL_AVG)
        cell.value        = round(float(avg_val), 3) if pd.notna(avg_val) else None
        cell.number_format = NUM_FMT
        cell.font          = Font(bold=True, size=9)
        cell.fill          = fill
        cell.alignment     = center()
        cell.border        = thin_border()

        # Event × seed value cells
        for i, ev in enumerate(events_ord):
            evdf = bdf[bdf["event"] == ev]
            for j, seed in enumerate(SEEDS):
                col   = COL_DATA + i * len(SEEDS) + j
                sdf   = evdf[evdf["seed_set"] == seed]
                val   = float(sdf[mcol].values[0]) if len(sdf) > 0 else None
                cell  = ws.cell(row=row, column=col)
                cell.value         = round(val, 3) if val is not None else None
                cell.number_format = NUM_FMT
                cell.font          = Font(size=9)
                cell.fill          = fill
                cell.alignment     = center()
                cell.border        = thin_border()

# ── Apply borders to header cells (after merge) ───────────────────────────────
for r in [1, 2]:
    for c in range(1, TOTAL_COLS + 1):
        cell = ws.cell(row=r, column=c)
        cell.border = thin_border()

# ── Column widths & row heights ───────────────────────────────────────────────
ws.column_dimensions[get_column_letter(COL_MODEL)].width  = 9
ws.column_dimensions[get_column_letter(COL_BUDGET)].width = 13
ws.column_dimensions[get_column_letter(COL_METRIC)].width = 10
ws.column_dimensions[get_column_letter(COL_AVG)].width    = 9

for i in range(N_EVENTS * len(SEEDS)):
    ws.column_dimensions[get_column_letter(COL_DATA + i)].width = 6.5

ws.row_dimensions[1].height = 32
ws.row_dimensions[2].height = 18
for r in range(HEADER_ROWS + 1, TOTAL_ROWS + 1):
    ws.row_dimensions[r].height = 16

# ── Save ──────────────────────────────────────────────────────────────────────
wb.save(OUTPUT_PATH)
print(f"Saved: {OUTPUT_PATH}")
print(f"  {N_EVENTS} events  ×  {len(SEEDS)} seeds  ×  {len(BUDGETS)} budgets  ×  {len(METRICS)} metrics")
print(f"  Sheet dimensions: {TOTAL_ROWS} rows × {TOTAL_COLS} columns")


Saved: ../results/cotrain_results_summary.xlsx
  9 events  ×  3 seeds  ×  4 budgets  ×  2 metrics
  Sheet dimensions: 10 rows × 31 columns
